In [ ]:
import pandas as pd
import numpy as np

def calculate_mortgage_term(principal, interest_rate, term_years, amortization_years, payment_frequency=12):
  """Calculates mortgage payments for a specific term.

  Args:
    principal: The initial mortgage amount.
    interest_rate: The annual interest rate as a decimal (e.g., 0.05 for 5%).
    term_years: The length of the mortgage term.
    amortization_years: The total amortization period.
    payment_frequency: The number of payments per year (default is 12 for monthly).

  Returns:
    A tuple containing a pandas DataFrame with payments and total amount paid and remaining balance.
  """

  # Calculate monthly interest rate and total number of payments
  monthly_interest_rate = interest_rate / payment_frequency
  total_payments = amortization_years * payment_frequency
  term_payments = term_years * payment_frequency

  # Calculate monthly payment based on full amortization
  monthly_payment = (principal * monthly_interest_rate) / (1 - (1 + monthly_interest_rate)**(-total_payments))

  # Create a DataFrame to store payment information
  mortgage_data = pd.DataFrame(columns=['Payment Number', 'Payment Amount', 'Interest Paid', 'Principal Paid', 'Remaining Balance'])

  # Initialize variables for total paid and remaining balance
  total_paid = 0
  remaining_balance = principal

  # Calculate payments for the specified term
  for i in range(1, term_payments + 1):
    interest_paid = remaining_balance * monthly_interest_rate
    principal_paid = monthly_payment - interest_paid
    remaining_balance -= principal_paid
    total_paid += monthly_payment

    mortgage_data = pd.concat([mortgage_data, pd.DataFrame({'Payment Number': [i],
                                         'Payment Amount': [monthly_payment],
                                         'Interest Paid': [interest_paid],
                                         'Principal Paid': [principal_paid],
                                         'Remaining Balance': [remaining_balance]})], ignore_index=True)

  return mortgage_data, total_paid, remaining_balance, monthly_payment

# Example usage
principal1 = 510000
interest_rate1 = 0.0485  # 4% interest rate
term_years1 = 5
amortization_years1 = 30

mortgage1, total_paid1, remaining_balance1, monthly_payment1 = calculate_mortgage_term(principal1, interest_rate1, term_years1, amortization_years1)

#print(mortgage1.tail())
print("Total amount paid:", round(total_paid1))
print("Remaining balance:", round(remaining_balance1))


# Example usage
principal1 = 500000
interest_rate1 = 0.0485  # 4% interest rate
term_years1 = 5
amortization_years1 = 25

mortgage2, total_paid2, remaining_balance2, monthly_payment2 = calculate_mortgage_term(principal1, interest_rate1, term_years1, amortization_years1)

#print(mortgage1.tail())
print("Total amount paid:", round(total_paid2))
print("Remaining balance:", round(remaining_balance2))
ahead_by=total_paid1-total_paid2+remaining_balance1-remaining_balance2
print("Ahead by:", round(ahead_by))
print(f'Mor 1: {round(monthly_payment1)}')
print(f'Mor 2: {round(monthly_payment2)}')

Total amount paid: 161474
Remaining balance: 467321
Total amount paid: 172765
Remaining balance: 441834
Ahead by: 14196
Mor 1: 2691
Mor 2: 2879


<ipython-input-1-8988956d5ad3>:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mortgage_data = pd.concat([mortgage_data, pd.DataFrame({'Payment Number': [i],
<ipython-input-1-8988956d5ad3>:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mortgage_data = pd.concat([mortgage_data, pd.DataFrame({'Payment Number': [i],


In [ ]:
total_paid2+remaining_balance2

614599.0050914073

In [ ]:
total_paid1+remaining_balance1

582524.9271399488

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def calculate_mortgage_term_variable(principal, interest_rates, term_years, amortization_years, start_date, payment_frequency=12):
  """Calculates mortgage payments for a specific term with variable interest rates.

  Args:
    principal: The initial mortgage amount.
    interest_rates: A dictionary of interest rates with their start dates.
    term_years: The length of the mortgage term.
    amortization_years: The total amortization period.
    start_date: The start date of the mortgage.
    payment_frequency: The number of payments per year (default is 12 for monthly).

  Returns:
    A pandas DataFrame with payments, total amount paid, and remaining balance.
  """

  # Convert interest rates to a list of tuples (start_date, interest_rate)
  interest_rate_data = sorted(interest_rates.items())

  # Convert start dates to datetime objects
  interest_rate_data = [(datetime.strptime(date_str, '%Y-%m-%d'), rate) for date_str, rate in interest_rate_data]

  # Calculate total payments and monthly payment based on initial interest rate
  total_payments = amortization_years * payment_frequency
  initial_interest_rate = interest_rate_data[0][1]
  monthly_interest_rate = initial_interest_rate / payment_frequency
  monthly_payment = (principal * monthly_interest_rate) / (1 - (1 + monthly_interest_rate)**(-total_payments))

  # Create a DataFrame to store payment information
  mortgage_data = pd.DataFrame(columns=['Payment Date', 'Payment Amount', 'Interest Paid', 'Principal Paid', 'Remaining Balance'])

  # Initialize variables
  remaining_balance = principal
  total_paid = 0
  payment_date = start_date

  # Iterate through interest rate periods
  for i, (start_date, interest_rate) in enumerate(interest_rate_data):
    monthly_interest_rate = interest_rate / payment_frequency
    end_date = interest_rate_data[i + 1][0] if i < len(interest_rate_data) - 1 else start_date + timedelta(days=365 * term_years)
    days_in_period = (end_date - start_date).days

    # Calculate number of payments in the period
    payments_in_period = int(days_in_period / (365 / payment_frequency))

    for j in range(payments_in_period):
      interest_paid = remaining_balance * monthly_interest_rate
      principal_paid = monthly_payment - interest_paid
      remaining_balance -= principal_paid
      total_paid += monthly_payment

      mortgage_data = pd.concat([mortgage_data, pd.DataFrame({'Payment Date': [payment_date],
                                                        'Payment Amount': [monthly_payment],
                                                        'Interest Paid': [interest_paid],
                                                        'Principal Paid': [principal_paid],
                                                        'Remaining Balance': [remaining_balance]})], ignore_index=True)
      payment_date += timedelta(days=365 / payment_frequency)

  return mortgage_data, total_paid, remaining_balance

# Example usage
principal = 510000
interest_rates = {'2024-08-31': 0.045, '2025-03-23': 0.040}
start_date = datetime(2024, 8, 31)
term_years = 3
amortization_years = 25

mortgage, total_paid, remaining_balance = calculate_mortgage_term_variable(principal, interest_rates, term_years, amortization_years, start_date)

#print(mortgage.tail())
print("Total amount paid:", round(total_paid))
print("Remaining balance:", round(remaining_balance))

# Example usage
principal1 = 510000
interest_rate1 = 0.0489  # 4% interest rate
term_years1 = 3
amortization_years1 = 25

mortgage1, total_paid1, remaining_balance1 = calculate_mortgage_term(principal1, interest_rate1, term_years1, amortization_years1)

#print(mortgage1.tail())
print("Total amount paid:", round(total_paid1))
print("Remaining balance:", round(remaining_balance1))


# Example usage
principal1 = 510000
interest_rate1 = 0.0454  # 4% interest rate
term_years1 = 5
amortization_years1 = 25

mortgage2, total_paid2, remaining_balance2 = calculate_mortgage_term(principal1, interest_rate1, term_years1-2, amortization_years1)

#print(mortgage1.tail())
print("Total amount paid:", round(total_paid2))
print("Remaining balance:", round(remaining_balance2))
ahead_by=total_paid1-total_paid2+remaining_balance1-remaining_balance2
print("Ahead by:", round(ahead_by))
print()

Total amount paid: 119059
Remaining balance: 460377
Total amount paid: 106157
Remaining balance: 476318
Total amount paid: 102468
Remaining balance: 474712
Ahead by: 5295

